<a href="https://colab.research.google.com/github/haroonwaheed19/Decision-Tree-Classifier-from-Scratch/blob/main/Decision_Tree_Classifier_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

In [ ]:
class Node:
  def __init__(self,feature_index=None,threshold=None,left=None,right=None,info_gain=None,value=None):
    #for decision node
    self.feature_index = feature_index
    self.threshold = threshold
    self.left = left
    self.right = right
    self.info_gain = info_gain

    #for leaf node
    self.value = value

In [ ]:
class DecisionTreeClassifier:
  #Constructor
  def __init__(self,min_samples_split=2,max_depth=2):
    #initializing the root of tree
    self.root = None

    #stopping conditions for our model
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth

  #Model training function
  def fit(self,X,Y):
    dataset = np.concatenate((X,Y),axis=1)
    self.root = self.build_tree(dataset)

  #helper function to build the tree
  def build_tree(self,dataset,curr_depth=0):
    X,Y = dataset[:,:-1],dataset[:,-1]

    num_samples,num_features = np.shape(X)

    #split untill stopping conditions are met
    if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
      #finding the best split for division of dataset
      best_split = self.get_best_split(dataset,num_samples,num_features)
      #checking if info_gain is positive or not
      if best_split["info_gain"] > 0:
        #recursively create left tree
        left_subtree = self.build_tree(best_split["dataset_left"],curr_depth + 1)
        #recursively create right tree
        right_subtree = self.build_tree(best_split["dataset_right"],curr_depth + 1)
        #return decision node
        return Node(best_split["feature_index"],best_split["threshold"],left_subtree,right_subtree,best_split["info_gain"])
    #calculate leaf node
    leaf_value = self.calculate_leaf_value(Y)

    #return leaf node
    return Node(value=leaf_value)

  #calculate leaf function
  def calculate_leaf_value(self,Y):
    Y = list(Y)
    return max(Y,key=Y.count)

  #get best split helper function
  def get_best_split(self,dataset,num_samples,num_features):
    #empty directory to store best split
    best_split = {}
    #initialize with negative infinity
    max_info_gain = -float("inf")

    #looping through over the features
    for feature_index in range(num_features):
      feature_values = dataset[:,feature_index]
      possible_thresholds = np.unique(feature_values)

      #looping through over the possible thresholds
      for threshold in possible_thresholds:
        #getting the left-node and right-node data
        #get current split
        dataset_left,dataset_right = self.split(dataset,feature_index,threshold)

        #checking if childs are not empty
        if len(dataset_left) > 0 and len(dataset_right) > 0:
          y,left_y,right_y = dataset[:,-1],dataset_left[:,-1],dataset_right[:,-1]

        #calcualting the information gain
        current_info_gain = self.information_gain(y,left_y,right_y,"gini")

        #updating the best split directory if the current gain is higher
        if(current_info_gain > max_info_gain):
          best_split["feature_index"] = feature_index
          best_split["threshold"] = threshold
          best_split["dataset_left"] = dataset_left
          best_split["dataset_right"] = dataset_right
          best_split["info_gain"] = current_info_gain
          max_info_gain = current_info_gain

    #return best split
    return best_split


  #split function for dataset
  def split(self,dataset,feature_index,threshold):
    dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])

    dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
    return dataset_left,dataset_right

  #information gain function
  def information_gain(self,parent,l_child,r_child,mode="entropy"):
    weight_l = len(l_child) / len(parent)
    weight_r = len(r_child) / len(parent)

    if mode == "gini":
      gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
    else:
      gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
    return gain

  #entropy function
  def entropy(self,y):
    class_labels = np.unique(y)
    entropy = 0
    for cls in class_labels:
      p_cls = len(y[y == cls]) / len(y)
      entropy += -p_cls * np.log2(p_cls)
    return entropy

  #Gini Impurity
  def gini_index(self,y):
    class_labels = np.unique(y)
    gini = 0
    for cls in class_labels:
      p_cls = len(y[y == cls]) / len(y)
      gini += p_cls ** 2
    return 1 - gini


  #Print Tree
  def print_tree(self,tree=None,indent=" "):
    if not tree:
      tree = self.root

    if tree.value is not None:
      print(tree.value)
    else:
      print("X_"+str(tree.feature_index),"<=",tree.threshold,":",tree.info_gain)
      print("%sleft:" % (indent),end="")
      self.print_tree(tree.left,indent + indent)
      print("%sright:" % (indent),end="")
      self.print_tree(tree.right,indent + indent)


  #predict function
  def predict(self,X):
    predictions = [self.make_prediction(x,self.root) for x in X]
    return predictions

  #Make prediction function
  def make_prediction(self,x,tree):
    if tree.value != None:
      return tree.value

    feature_val = x[tree.feature_index]
    if feature_val <= tree.threshold:
      return self.make_prediction(x,tree.left)
    else:
      return self.make_prediction(x,tree.right)